# Channel network and REW delineation 

In this notebook, we'll cover the steps required to extract the REW channel network. This tutorial is specific to GRASS 7.0 and the Mac operating system (OS X). We have been most succesful with and recommend installing the GRASS 7.0.x files maintained by Michael Barton, which can be found [here](https://grass.osgeo.org/download/software/mac-osx/). Newer versions of GRASS (`7.3.x`) exist, but have not been tested for this tutorial. NOTE: For newer versions of OS X (El Capitan and on), users may need to implement [this](http://grassmac.wikidot.com) fix to get GRASS to run sucesfully. 

In [1]:
%matplotlib inline
from matplotlib import pylab
import os
import sys
from os.path import dirname
import pickle
import gdal
import fiona
import glob
import shapely
from shapely import geometry,ops
import numpy as np
import fiona
import shapely
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import geopandas as gp
import seaborn
import pyproj
parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','1_data_preparation'))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','2_hillslope_discharge'))
from vadoseZone import LaioVadoseZone, PorporatoVadoseZone
from groundwaterZone import GroundwaterZone, NonlinearReservoir, NonlinearReservoir, TwoLinearReservoir, TwoParallelLinearReservoir

/Users/daviddralle/anaconda2/envs/py2k_gis/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Clip DEM

First, we would like to make sure that our digital elevation model (DEM) is clipped to the modeling region. This step is necessary due to the fact that the elevation file currently in the `raw_data/dem` folder is a full National Elevation Dataset (NED) tile, which is much larger than the computational domain of our watershed. 

In [2]:
%%bash

# Export again, otherwise GDAL won't be able to access projections list. 
# Necessary environment variables must be exported every new cell 
# because the %%bash command restarts a new bash kernel every time
export PATH=/Library/Frameworks/GDAL.framework/Programs:$PATH
export PROJSO=/Library/Frameworks/PROJ.framework/PROJ

#define the path of the full tile DEM, as well as the clipping polygon
export MASTER_DEM=../../raw_data/dem/full_tile.tif
export WATERSHED_POLY=../../raw_data/watershed_poly/sf_miranda.shp
rm ../../raw_data/dem/dem.tif
gdalwarp -cutline $WATERSHED_POLY -crop_to_cutline $MASTER_DEM ../../raw_data/dem/dem.tif


Creating output file that is 5825P x 7113L.
Processing input file ../../raw_data/dem/full_tile.tif.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Extracting the REWs

At this point, we're ready to perform our network extraction analysis on the DEM. We will do all of this operating GRASS from a BASH command line. The particulars here are a bit messy, but essentially we create a temporary workspace for GRASS, open the DEM, perform the extraction, then close everything and cleanup. The main script used for extracting the network can be found in `1_data_preparation/extract_stream_basins_topology.sh`.

In [3]:
%%bash

# make the script executable for GRASS 
chmod u+x ../1_data_preparation/extract_stream_basins_topology.sh

# create folder to store temporary grass location
DBASE=../../grassdata
mkdir $DBASE

# setup GRASS setup file to find database. This step is necessary when using GRASS from the command line. 
echo "LOCATION_NAME: 
GISDBASE:$DBASE
MAPSET: 
GUI: 
PID:" > $HOME/.grass7/rc

# create new temporary location for the job, exit after creation of this location
/Applications/GRASS-7.0.app/Contents/MacOS/grass.sh -text -c ../../raw_data/dem/dem.tif ../../grassdata/mylocation
 

Rebuilding Addon HTML manual pages index...
Rebuilding Addon menu...
Python 2.7.10 found.


Cleaning up temporary files...
Creating new GRASS GIS location/mapset...

          __________  ___   __________    _______________
         / ____/ __ \/   | / ___/ ___/   / ____/  _/ ___/
        / / __/ /_/ / /| | \__ \\_  \   / / __ / / \__ \ 
       / /_/ / _, _/ ___ |___/ /__/ /  / /_/ // / ___/ / 
       \____/_/ |_/_/  |_/____/____/   \____/___//____/  

Welcome to GRASS GIS 7.0.4
GRASS GIS homepage:                      http://grass.osgeo.org
This version running through:            Bash Shell (/bin/bash)
Help is available with the command:      g.manual -i
See the licence terms with:              g.version -c
Start the GUI with:                      g.gui wxpython
When ready to quit enter:                exit

Cleaning up temporary files...
Done.

Goodbye from GRASS GIS



Run the GRASS batch job to extract REWs and network connectivity. 

In [4]:
%%bash

# define the job file as environmental variable
export GRASS_BATCH_JOB="../1_data_preparation/extract_stream_basins_topology.sh"

# define channel head formation threshold; make it rather large to decrease
# computational demands later
export THRESH=500

# must set path to model folder for .sh script so it can find necessary files and store output
export MODEL=../../

# set name of polygon defining watershed
# must be the same as the clipping polygon above
export WATERSHED_POLY=../../raw_data/watershed_poly/sf_miranda.shp
 
# now we can use the temp location and run the job defined via GRASS_BATCH_JOB
/Applications/GRASS-7.0.app/Contents/MacOS/grass.sh ../../grassdata/mylocation/PERMANENT
 
# CLEANUP
unset GRASS_BATCH_JOB
 
# delete temporary location
rm -rf ../../grassdata/

# clear out rc file so that the GRASS GUI opens properly on next opening
echo "LOCATION_NAME: 
GISDBASE:
MAPSET: 
GUI: 
PID:" > $HOME/.grass7/rc

Rebuilding Addon HTML manual pages index...
Rebuilding Addon menu...
Python 2.7.10 found.
projection: 3 (Latitude-Longitude)
zone:       0
datum:      nad83
ellipsoid:  grs80
north:      40:15:41.208876N
south:      39:36:10.13148N
west:       123:57:53.148528W
east:       123:25:31.574028W
nsres:      0:00:00.333344
ewres:      0:00:00.333318
rows:       7113
cols:       5825
cells:      41433225
projection: 3 (Latitude-Longitude)
zone:       0
datum:      nad83
ellipsoid:  grs80
north:      40:15:45N
south:      39:36:10N
west:       123:57:55W
east:       123:25:30W
nsres:      0:00:05
ewres:      0:00:05
rows:       475
cols:       389
cells:      184775


Cleaning up temporary files...
Starting GRASS GIS...
Executing '../1_data_preparation/extract_stream_basins_topology.sh' ...
Proceeding with import of 1 raster bands...
Importing raster map <dem_unfilled>...
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42%  45%  48%  51%  54%  57%  60%  63%  66%  69%  72%  75%  78%  81%  84%  87%  90%  93%  96%  99% 100%
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42%  45%  48%  51%  54%  57%  60%  63%  66%  69%  72%  75%  78%  81%  84%  87%  90%  93%  96%  99% 100%
SECTION 1a (of 4): Initiating Memory.
SECTION 1b (of 4): Determining Offmap Flow.
   0%

## Extracted REWs

Let's take a look at the REWs extracted using the above script. Here, we import the polygons corresponding to each REW, and map these REWs with labels corresponding to the REW identification number (which will be used extensively throughout these tutorials). 

In [5]:
pylab.rcParams['figure.figsize'] = (26, 18)
basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
basins_shape = gp.GeoDataFrame.from_file(basins)
basins_shape['coords'] = basins_shape['geometry'].apply(lambda x: x.representative_point().coords[:])
basins_shape['coords'] = [coords[0] for coords in basins_shape['coords']]

ax1 = basins_shape.plot()
for idx, row in basins_shape.iterrows():
    print_str = 'REW' + str(row['cat'])
    plt.annotate(s=print_str, xy=row['coords'],
                 horizontalalignment='center',fontsize=15)

streams = glob.glob(os.path.join(parent_dir,'raw_data','streams_poly','*.shp'))[0]
streams_shape = gp.GeoDataFrame.from_file(streams)
streams_shape.plot(ax=ax1,color='blue')

ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
ax1.patch.set_facecolor('white')


KeyError: 'geometry'